# Converting a NetCDF File to GeoJSON

A function that takes a NetCDF file and converts it into a GeoJSON collection of points. Libraries required: geojson, xarray, numpy.

General approach:
1. Import libraries and read in NetCDF file.
2. Make new longitude/latitude coordinate arrays between (-180, 180) for longitude and (-90, 90) for latitude to make the data fit an equidistant cylindrical map projection (EPSG:4087).
3. Loop over coordinates to create a GeoJSON feature point for each pair; append to create GeoJSON file.

In [ ]:
import geojson
import xarray as xr
import numpy as np

def netcdf_to_geojson(nc_file, lon_variable, lat_variable, data_variable, geojson_name):
    """
    Returns: a GeoJSON file created from the data in a structured NetCDF file, with coordinates following an 
    equidistant cylindrical projection (EPSG:4087) and a property storing one specified data variable.
    
    Parameters:
    nc_file: String. Path and name of structured NetCDF file to convert.
    lon_variable: String. Name of coordinate variable used to represent the data on a horizontal basis in the NetCDF file (i.e. 'lon', 'x').
    lat_variable: String. Name of coordinate variable used to represent the data on a vertical basis in the NetCDF file (i.e. 'lat', 'y').
    data_variable: String. Name of data variable of interest from the NetCDF file. Will be retained as a property of the GeoJSON file.
    geojson_name: String. Path and name of desired GeoJSON file output.
    """
    # open dataset
    ds = xr.open_dataset(nc_file)
    
    # "reprojecting" by making new coordinate arrays following the equidistant cylindrical projection
    step_lon = 360 / ds[lon_variable].size
    step_lat = 180 / ds[lat_variable].size

    reproject_lon = np.arange(-180, 180, step_lon)
    reproject_lat = np.arange(-90, 90, step_lat)

    new_latitude_var = xr.DataArray(reproject_lat, dims="latitude")
    new_longitude_var = xr.DataArray(reproject_lon, dims="longitude")
    
    # update the dataset with new coordinate variables
    ds["latitude"] = new_latitude_var
    ds["longitude"] = new_longitude_var

    # identify key variables
    latitude = ds["latitude"]
    latitude_range = len(latitude)
    longitude = ds["longitude"]
    longitude_range = len(longitude)
    data_var = ds[data_variable].round(0)
    
    # prepare GeoJSON
    geojson_features = []
    
    # loop over all coordinate pairs
    for i in range(latitude_range):
        for j in range(longitude_range):
            lat = float(latitude[i])
            lon = float(longitude[j])
            data_value = float(data_var[i, j])

            # Create a GeoJSON feature for each point
            point = geojson.Point((lon, lat))
            properties = {data_variable: data_value}
            feature = geojson.Feature(geometry=point, properties=properties)
            geojson_features.append(feature)
    
    # write GeoJSON to File
    with open(geojson_name, "w") as f:
        geojson.dump(geojson.FeatureCollection(geojson_features), f)

In [ ]:
netcdf_to_geojson("../sample_data/h1.2000-11-15-10800.t0_360x720.nc", "x", "y", "IVT", "../geojson_data/360x720.geojson")

In [ ]:
# larger files may require several minutes to run.
# commented out line below due to the resulting file being above GitHub upload size limits.
# netcdf_to_geojson("../sample_data/h1.2000-11-15-10800.t0_360x720.nc", "x", "y", "IVT", "../geojson_data/720x1440.geojson")